In [1]:
import os

class Process:
    def __init__(self, pid, arrival_time, burst_time):
        self.pid = pid
        self.arrival_time = arrival_time
        self.burst_time = burst_time
        self.remaining_time = burst_time
        self.start_time = 0
        self.finish_time = 0
        self.waiting_time = 0
        self.turnaround_time = 0
        self.executed = False


In [2]:
def read_processes_from_file(file_path):
    processes = []
    with open(file_path, 'r') as file:
        for line in file:
            if line.strip():
                pid, arrival_time, burst_time = map(int, line.split())
                processes.append(Process(pid, arrival_time, burst_time))
    return processes


In [3]:
def fcfs_scheduler(processes):
    current_time = 0
    for process in processes:
        if current_time < process.arrival_time:
            current_time = process.arrival_time
        process.start_time = current_time
        process.finish_time = current_time + process.burst_time
        process.turnaround_time = process.finish_time - process.arrival_time
        process.waiting_time = process.turnaround_time - process.burst_time
        current_time = process.finish_time


In [4]:
def sjf_scheduler(processes):
    current_time = 0
    ready_queue = []
    finished_processes = []
    processes.sort(key=lambda x: x.arrival_time)

    while len(finished_processes) < len(processes):
        for process in processes:
            if not process.executed and process.arrival_time <= current_time:
                ready_queue.append(process)
                process.executed = True

        ready_queue.sort(key=lambda x: x.burst_time)

        if ready_queue:
            process = ready_queue.pop(0)
            if current_time < process.arrival_time:
                current_time = process.arrival_time
            process.start_time = current_time
            process.finish_time = current_time + process.burst_time
            process.turnaround_time = process.finish_time - process.arrival_time
            process.waiting_time = process.start_time - process.arrival_time
            current_time = process.finish_time
            finished_processes.append(process)
        else:
            next_process = next((p for p in processes if not p.executed), None)
            if next_process:
                current_time = next_process.arrival_time
    return finished_processes


In [5]:
def round_robin_scheduler(processes, quantum):
    current_time = 0
    ready_queue = processes[:]
    finished_processes = []

    while ready_queue:
        process = ready_queue.pop(0)
        if process.remaining_time > quantum:
            current_time += quantum
            process.remaining_time -= quantum
            ready_queue.append(process)
        else:
            current_time += process.remaining_time
            process.remaining_time = 0
            process.finish_time = current_time
            process.turnaround_time = process.finish_time - process.arrival_time
            process.waiting_time = process.turnaround_time - process.burst_time
            finished_processes.append(process)
    return finished_processes


In [6]:
def display_results(processes):
    total_waiting_time = 0
    total_turnaround_time = 0
    total_cpu_time = sum(p.burst_time for p in processes)

    print(f"{'Process ID':<10}{'Finish Time':<15}{'Waiting Time':<15}{'Turnaround Time':<15}")
    for process in processes:
        print(f"{process.pid:<10}{process.finish_time:<15}{process.waiting_time:<15}{process.turnaround_time:<15}")
        total_waiting_time += process.waiting_time
        total_turnaround_time += process.turnaround_time

    cpu_utilization = (total_cpu_time / processes[-1].finish_time) * 100
    print(f"\nTotal CPU Utilization: {cpu_utilization:.2f}%")
    print(f"Average Waiting Time: {total_waiting_time / len(processes):.2f}")
    print(f"Average Turnaround Time: {total_turnaround_time / len(processes):.2f}")


In [17]:
def main():
    file_path = input("Enter the path to the file ( sample_processes.txt): ").strip()
    if not os.path.exists(file_path):
        print("File not found.")
        return

    processes = read_processes_from_file(file_path)

    print("Choose a scheduling algorithm:")
    print("1. FCFS (First-Come, First-Served)")
    print("2. SJF (Shortest Job First)")
    print("3. Round Robin")

    choice = input("Enter the number of the algorithm you want to run: ").strip()
    if choice == '1':
        fcfs_scheduler(processes)
        print('\nFCFS Scheduling Results:')
        display_results(processes)
    elif choice == '2':
        sjf_finished_processes = sjf_scheduler(processes)
        print('\nSJF Scheduling Results:')
        display_results(sjf_finished_processes)
    elif choice == '3':
        quantum = int(input("Enter the quantum time (1,2,3, 4, ..etc): ").strip())
        rr_finished_processes = round_robin_scheduler(processes, quantum)
        print(f'\nRound Robin Scheduling Results (Quantum = {quantum}):')
        display_results(rr_finished_processes)
    else:
        print("Invalid choice. Please choose a valid option (1, 2, or 3).")


In [36]:
if __name__ == "__main__":
    main()


Enter the path to the file ( sample_processes.txt): sample_processes.txt
Choose a scheduling algorithm:
1. FCFS (First-Come, First-Served)
2. SJF (Shortest Job First)
3. Round Robin
Enter the number of the algorithm you want to run: 3
Enter the quantum time (1,2,3, 4, ..etc): 5

Round Robin Scheduling Results (Quantum = 5):
Process IDFinish Time    Waiting Time   Turnaround Time
2         9              5              9              
4         19             11             16             
1         22             14             22             
3         26             15             24             

Total CPU Utilization: 100.00%
Average Waiting Time: 11.25
Average Turnaround Time: 17.75
